In [1]:
import os
import numpy as np
import pandas as pd
import re

In [2]:
li_pos = os.listdir('train/pos')

In [3]:
len_li_pos = len(li_pos)

In [4]:
li_neg = os.listdir('train/neg')

In [5]:
len_li_neg = len(li_neg)

In [6]:
df = pd.DataFrame()

In [7]:
df['sentence'] = ''
df['sentiment'] = ''
df['polarity'] = ''

In [8]:
for i in range(len_li_pos + len_li_neg):
    if i < len_li_pos :
        sentence = open('train/pos/'+li_pos[i], encoding = 'utf-8').read()
        sentiment = re.match("\d+_(\d+)\.txt", li_pos[i]).group(1)
        polarity = 1
        df.loc[i] = [sentence, sentiment, polarity]
    else:
        sentence = open('train/neg/'+li_neg[i - len_li_pos], encoding = 'utf-8').read()
        sentiment = re.match("\d+_(\d+)\.txt", li_neg[i - len_li_pos]).group(1)
        polarity = 0
        df.loc[i] = [sentence, sentiment, polarity]

In [9]:
df.head()

,sentence,sentiment,polarity
0,Bromwell High is a cartoon comedy. It ran at t...,9,1
1,Homelessness (or Houselessness as George Carli...,8,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,10,1
3,This is easily the most underrated film inn th...,7,1
4,This is not the typical Mel Brooks film. It wa...,8,1


In [10]:
df.tail()

,sentence,sentiment,polarity
24995,"Towards the end of the movie, I felt it was to...",4,0
24996,This is the kind of movie that my enemies cont...,3,0
24997,I saw 'Descent' last night at the Stockholm Fi...,3,0
24998,Some films that you pick up for a pound turn o...,1,0
24999,"This is one of the dumbest films, I've ever se...",1,0


In [11]:
df.shape

(25000, 3)

In [12]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [13]:
len(stop_words)

179

In [14]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [15]:
len(contractions)

120

In [16]:
modified_sentences = []
for i in range(len(df['sentence'])):
    words = df['sentence'][i].lower().split(' ')
    new_li = []
    for w in words:
        if w in contractions:
            new_li.append(contractions[w])
        else:
            new_li.append(w)
    
    for ele in new_li:
        if ele in stop_words:
            new_li.remove(ele)
            
    string = " ".join(new_li)
    
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', string, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub('[m]{2,}', 'mm', text)
    
    modified_sentences.append(text)

In [17]:
modified_sentences[:5]

['bromwell high cartoon comedy  ran time some programs school life  as  teachers   35 years teaching profession lead believe bromwell high s satire much closer reality  teachers   scramble survive financially  insightful students can see right their pathetic teachers  pomp  pettiness the whole situation  remind the schools knew their students  saw the episode which student repeatedly tried burn the school  immediately recalled                      high  a classic line  inspector  i am to sack one your teachers  student  welcome to bromwell high  i expect many adults of my age think bromwell high is far fetched  a pity it isn t ',
 'homelessness  or houselessness george carlin stated  been issue years never plan help street were considered human everything going school  work  vote matter  people think homeless just lost cause worrying things racism  war iraq  pressuring kids succeed  technology  elections  inflation  worrying they will next end streets <br  ><br  >but were given bet liv

In [18]:
len(modified_sentences)

25000

In [19]:
final_df = pd.DataFrame()
final_df['sentence'] = ''
final_df['sentiment'] = ''
final_df['polarity'] = ''

In [20]:
for i in range(len_li_pos + len_li_neg):
    if i < len_li_pos :
        sentence = modified_sentences[i]
        sentiment = re.match("\d+_(\d+)\.txt", li_pos[i]).group(1)
        polarity = 1
        final_df.loc[i] = [sentence, sentiment, polarity]
    else:
        sentence = modified_sentences[i]
        sentiment = re.match("\d+_(\d+)\.txt", li_neg[i - len_li_pos]).group(1)
        polarity = 0
        final_df.loc[i] = [sentence, sentiment, polarity]

In [21]:
final_df.head()

,sentence,sentiment,polarity
0,bromwell high cartoon comedy ran time some pr...,9,1
1,homelessness or houselessness george carlin s...,8,1
2,brilliant over acting lesley ann warren best ...,10,1
3,easily underrated film inn brooks cannon sure...,7,1
4,is typical mel brooks film much less slapstic...,8,1


In [22]:
final_df.tail()

,sentence,sentiment,polarity
24995,towards end movie felt technical felt like c...,4,0
24996,kind movie my enemies content watch time it i...,3,0
24997,saw descent last night stockholm film festiv...,3,0
24998,films you pick pound turn rather good 23rd c...,1,0
24999,one dumbest films i have ever seen rips near...,1,0


In [23]:
final_df.to_csv(r'C:/Users/Arshit/Text Classification/train_data.csv', header='sentence', index=None, sep=' ', mode='w')

In [24]:
final_df = pd.read_csv('train_data.csv', delimiter=' ')

In [25]:
final_df.head()

,sentence,sentiment,polarity
0,bromwell high cartoon comedy ran time some pr...,9,1
1,homelessness or houselessness george carlin s...,8,1
2,brilliant over acting lesley ann warren best ...,10,1
3,easily underrated film inn brooks cannon sure...,7,1
4,is typical mel brooks film much less slapstic...,8,1


In [26]:
final_df['sentence']

0        bromwell high cartoon comedy  ran time some pr...
1        homelessness  or houselessness george carlin s...
2        brilliant over acting lesley ann warren  best ...
3        easily underrated film inn brooks cannon  sure...
4        is typical mel brooks film  much less slapstic...
                               ...                        
24995    towards end movie  felt technical  felt like c...
24996    kind movie my enemies content watch time  it i...
24997    saw  descent  last night stockholm film festiv...
24998    films you pick pound turn rather good   23rd c...
24999    one dumbest films  i have ever seen  rips near...
Name: sentence, Length: 25000, dtype: object

In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(final_df['sentence'])

thresh=35

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value

common = tot_cnt-cnt

tokenizer = Tokenizer(num_words = common)
tokenizer.fit_on_texts(final_df['sentence'])
word2idx = tokenizer.word_index

In [29]:
len_word2idx = common

In [30]:
len_word2idx

9253

In [31]:
sequences = tokenizer.texts_to_sequences(final_df['sentence'])

In [32]:
len(sequences)

25000

In [33]:
X_train = pad_sequences(sequences, padding='post', maxlen=200)

In [34]:
X_train.shape

(25000, 200)

In [35]:
Y_train = []
for i in range(25000):
    if i < 12500:
        Y_train.append(1)
    else:
        Y_train.append(0)

In [36]:
len(Y_train)

25000

In [37]:
Y_train = np.reshape(Y_train, (len(Y_train),)+(1,))

In [38]:
Y_train.shape

(25000, 1)

In [39]:
test_li_pos = os.listdir('test/pos')
len_test_li_pos = len(test_li_pos)

test_li_neg = os.listdir('test/neg')
len_test_li_neg = len(test_li_neg)

test_df = pd.DataFrame()

test_df['sentence'] = ''
test_df['sentiment'] = ''
test_df['polarity'] = ''

for i in range(len_test_li_pos + len_test_li_neg):
    if i < len_test_li_pos :
        sentence = open('test/pos/'+test_li_pos[i], encoding = 'utf-8').read()
        sentiment = re.match("\d+_(\d+)\.txt", test_li_pos[i]).group(1)
        polarity = 1
        test_df.loc[i] = [sentence, sentiment, polarity]
    else:
        sentence = open('test/neg/'+test_li_neg[i - len_test_li_pos], encoding = 'utf-8').read()
        sentiment = re.match("\d+_(\d+)\.txt", test_li_neg[i - len_test_li_pos]).group(1)
        polarity = 0
        test_df.loc[i] = [sentence, sentiment, polarity]


modified_sentences = []
for i in range(len(test_df['sentence'])):
    words = test_df['sentence'][i].lower().split(' ')
    new_li = []
    for w in words:
        if w in contractions:
            new_li.append(contractions[w])
        else:
            new_li.append(w)
    
    for ele in new_li:
        if ele in stop_words:
            new_li.remove(ele)
            
    string = " ".join(new_li)
    
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', string, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub('[m]{2,}', 'mm', text)
    
    modified_sentences.append(text)
    

final_test_df = pd.DataFrame()

final_test_df['sentence'] = ''
final_test_df['sentiment'] = ''
final_test_df['polarity'] = ''

for i in range(len_test_li_pos + len_test_li_neg):
    if i < len_test_li_pos :
        sentence = modified_sentences[i]
        sentiment = re.match("\d+_(\d+)\.txt", test_li_pos[i]).group(1)
        polarity = 1
        final_test_df.loc[i] = [sentence, sentiment, polarity]
    else:
        sentence = modified_sentences[i]
        sentiment = re.match("\d+_(\d+)\.txt", test_li_neg[i - len_test_li_pos]).group(1)
        polarity = 0
        final_test_df.loc[i] = [sentence, sentiment, polarity]

In [55]:
tokenizer_test = Tokenizer(num_words = common)
tokenizer_test.fit_on_texts(final_test_df['sentence'])
word2idx_test = tokenizer_test.word_index
sequences_test = tokenizer_test.texts_to_sequences(final_test_df['sentence'])

X_test = pad_sequences(sequences_test, padding='post', maxlen=200)

Y_test = []
for i in range(25000):
    if i < 12500:
        Y_test.append(1)
    else:
        Y_test.append(0)
        
Y_test = np.reshape(Y_test, (len(Y_test),)+(1,))

In [41]:
X_test.shape

(25000, 200)

In [42]:
Y_test.shape

(25000, 1)

In [43]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import random
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [44]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [45]:
from tensorflow.keras import layers

In [46]:
input_X = keras.Input(shape=(200,))

X = layers.Embedding(len_word2idx, 128)(input_X)
X = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(X)
X = layers.Bidirectional(layers.LSTM(64))(X)

out = layers.Dense(1, activation='sigmoid')(X)

model = keras.Model(input_X, out)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [47]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 128)          1184384   
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 128)          98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,382,145
Trainable params: 1,382,145
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [49]:
model.fit(X_train, Y_train, epochs=5, batch_size = 64)

Train on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 512s 20ms/sample - loss: 0.3838 - acc: 0.8262
Epoch 2/5
25000/25000 [==============================] - 333s 13ms/sample - loss: 0.2173 - acc: 0.9195
Epoch 3/5
25000/25000 [==============================] - 340s 14ms/sample - loss: 0.1547 - acc: 0.9436
Epoch 4/5
25000/25000 [==============================] - 333s 13ms/sample - loss: 0.1117 - acc: 0.9612
Epoch 5/5
25000/25000 [==============================] - 334s 13ms/sample - loss: 0.0851 - acc: 0.9708


In [50]:
train_loss, train_acc = model.evaluate(X_train, Y_train)

25000/25000 [==============================] - 44s 2ms/sample - loss: 0.0454 - acc: 0.9898


In [51]:
print("train_loss : ", train_loss)
print("train_acc : ", train_acc)

train_loss :  0.04535048822373152
train_acc :  0.9898


In [52]:
pred_train = model.predict(X_train)

In [56]:
test_loss, test_acc = model.evaluate(X_test, Y_test)

25000/25000 [==============================] - 47s 2ms/sample - loss: 1.5816 - acc: 0.5396


In [57]:
print("test_loss : ", test_loss)
print("test_acc : ", test_acc)

test_loss :  1.5815725871276856
test_acc :  0.53964


In [58]:
pred_test = model.predict(X_test)